In [2]:
import spacy
import nltk
import string
import numpy as np

In [7]:
!python3 -m spacy download en_core_web_lg
import en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.8 MB/s eta 0:00:0000:0100:04

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
import importlib
import embedding_search as es
importlib.reload(es)

<module 'embedding_search' from '/opt/homebrew/lib/python3.11/site-packages/embedding_search.cpython-311-darwin.so'>

In [4]:
nlp = spacy.load('en_core_web_lg')

In [5]:
def preprocess(chunk: str):
    chunk = chunk.lower()
    chunk = chunk.translate(str.maketrans('', '', string.punctuation))
    return chunk

def get_vector(chunk: str, nlp):
    preprocessed = preprocess(chunk)
    token_embeddings = []
    for token in preprocessed:
        token_embeddings.append(nlp(token).vector)
    return np.mean(token_embeddings, axis=0)

In [6]:
# prepare sample data
# data = open("interstellar_raw.txt").read()
# LINES_PER_CHUNK = 3
# lines = data.split("\n")
# chunks = []
# for i in range(0, len(lines), LINES_PER_CHUNK):
#     chunk = " ".join(lines[i:i+LINES_PER_CHUNK])
#     # remove punctuation
#     chunk = preprocess(chunk)
#     chunks.append(chunk)

# tokenized_chunks = [nltk.word_tokenize(chunk) for chunk in chunks]

data = open("harry-potter-1.txt").read()
sentences = nltk.sent_tokenize(data)
preprocessed_sentences = [preprocess(sentence) for sentence in sentences]
tokenized_chunks = [nltk.word_tokenize(sentence) for sentence in preprocessed_sentences]
len(tokenized_chunks)

6394

In [7]:
# embeddings
embeddings = []
for chunk_list in tokenized_chunks:
    chunk_embeddings = []
    for chunk in chunk_list:
        vec = nlp(chunk).vector
        if np.all(vec == 0):
            continue
        chunk_embeddings.append(vec)
    embeddings.append(np.mean(chunk_embeddings, axis=0))
len(embeddings)

/opt/homebrew/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


6394

In [8]:
embeddings = [embedding for embedding in embeddings if np.isnan(embedding).any() == False]

In [9]:
store = es.EmbeddingStore("test/", 300, 1024 * 1024, 1024 * 1024)

In [10]:
for chunk, embedding in zip(tokenized_chunks, embeddings):
    try:
        store.addEmbedding(embedding, " ".join(chunk))
    except Exception as e:
        print(e)


In [30]:
search_key = get_vector("one step forward free to checkmate", nlp)
closest = store.getKClosest(search_key, 10, 1, es.EmbeddingStore.DistanceMetric.cosine_similarity)

In [31]:
closest

['ive written them a letter',
 'i know you haven t said professor mcgonagall sounding half exasperated half admiring',
 'he stayed at home and nailed up the mail slot',
 'i know',
 'whats this',
 'harry was on the point of unfolding his letter which was written on the same heavy parchment as the envelope when it was jerked sharply out of his hand by uncle vernon',
 'yes i quite agree',
 'even if i could i wouldnt',
 'hows that popkin',
 'the potters thats right thats what i heard yes their son harry mr dursley stopped dead']

In [29]:
for chunk in tokenized_chunks:
    if "checkmate" in chunk:
        exp_result = get_vector(" ".join(chunk), nlp)
        cos_sim = np.dot(search_key, exp_result) / (np.linalg.norm(search_key) * np.linalg.norm(exp_result))
        print(cos_sim)

0.9882876


In [34]:
returned_vec = get_vector(closest[0], nlp)
cos_sim = np.dot(search_key, returned_vec) / (np.linalg.norm(returned_vec) * np.linalg.norm(search_key))
print(cos_sim)

0.9460092
